In [1]:
import sqlalchemy
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.types import *
from sqlalchemy import create_engine
import os
import json
import time

In [2]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Soccer Data") \
    .getOrCreate()

In [11]:
# def fetch_matchday(self):
log_name = "matchday"
url = f"https://www.sofascore.com/api/v1/sport/football/scheduled-events/2023-07-31"
response = requests.get(url)
status_code = response.status_code

if status_code != 200:
    print(f"Error code: {status_code}. Stop jobs!")
else:
    match_in_day = response.json()
    match_in_day = match_in_day['events']

    # Normalize the nested JSON data
    df_matchday = pd.json_normalize(match_in_day)
    df_matchday.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    df_matchday.rename(columns={"id": "match_id"}, inplace=True)
    df_football = df_matchday[df_matchday['tournament_category_sport_name'] == 'Football']

    # Print the combined DataFrame
    df = pd.concat([df_football], ignore_index=True)
    print("Dataframe is here!!!")
    print(df)
    list_match_id = df["match_id"].astype(str).tolist()
    list_customId = df["customId"]
    # self.export(log_name, df)
    # Fetch and export shotmap data
    # shotmap_df = shotmap(list_match_id)
    # if shotmap_df is not None:
    #     self.export("shotmap", shotmap_df)

# def shotmap(self, list_match_id):
#     shotmap_data = []
# 
#     for match_id in list_match_id:
#         url = f"https://www.sofascore.com/api/v1/event/{match_id}/shotmap"
#         try:
#             response = requests.get(url)
#             status_code = response.status_code
#             if status_code == 200:
#                 data = response.json()
#                 for event in data:
#                     event['match_id'] = match_id  # Add match_id to each event
#                 shotmap_data.extend(data)
#             else:
#                 print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
#         except Exception as e:
#             print(f"An error occurred for match ID {match_id}: {e}")
#         time.sleep(1)
# 
#     if shotmap_data:
#         # Normalize the nested JSON data
#         df_shotmap = pd.json_normalize(shotmap_data)
#         df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#         return df_shotmap
#     else:
#         return None

Dataframe is here!!!
      customId  winnerCode  hasGlobalHighlights hasEventPlayerStatistics  \
0      kYcsmRc         2.0                False                    False   
1      rYcsDPi         3.0                False                    False   
2    Dwdbsbxpc         2.0                False                    False   
3      tYkswXo         1.0                 True                    False   
4      nYcopYc         2.0                False                    False   
..         ...         ...                  ...                      ...   
234    GokkUjn         1.0                False                      NaN   
235     EPsTka         3.0                 True                      NaN   
236    KTqsoGB         2.0                 True                      NaN   
237     UJsXdb         2.0                False                     True   
238   KrcsBRtb         3.0                 True                      NaN   

    hasEventPlayerHeatMap  detailId  crowdsourcingDataDisplayEnabl

In [9]:
print(list_match_id)

['12173508', '12173471', '12173472', '12173517', '12173516', '12172425', '12172419', '12172525', '12172526', '12172437', '12172434', '12172441', '12172453', '12172509', '12172501', '12172345', '12172348', '12172359', '12172356', '12172350', '12172339', '12172333', '12172328', '12172329', '12214127', '12213731', '12310661', '12310660', '12310659', '12310668', '12255885', '12255883', '12255882', '12255880', '12146236', '11457108', '12310473', '12060302', '12060303', '12060304', '12287635', '12097656', '12097663', '12097660', '12097664', '12004537', '11453188', '11453182', '11453194', '11453178', '12295020', '12295017', '12295018', '12295013', '12091457', '11378835', '12026342', '12026337', '12026344', '11424125', '11451575', '11451642', '11451574', '11451572', '11451573', '11451571', '12277222', '12277221', '12297504', '12297506', '12297496', '12297499', '12288965', '12288963', '12288962', '12288968', '12288964', '12288971', '12162795', '12162793', '12115569', '12264393', '12270392', '12

In [12]:
# list_match_id = ["12173508"]
event_data = []
for customId in list_customId:
    url = f"https://www.sofascore.com/api/v1/event/{customId}/h2h/events"
    try:
        response = requests.get(url)
        status_code = response.status_code
        if status_code == 200:
            data = response.json()
            for event in data['events']:
                event_copy = event.copy()
                event_copy['customId'] = customId
                event_copy['status_code'] = status_code
                event_data.append(event_copy)
        else:
            print(f"Failed to fetch data for match ID {customId}. Status code: {status_code}")
            event_data.append({'match_id': customId, 'status_code': status_code})
    except Exception as e:
        print(f"An error occurred for match ID {customId}: {e}")
        event_data.append({'match_id': customId, 'status_code': None})
    time.sleep(1)

if event_data:
    df_match_event = pd.json_normalize(event_data)
    df_match_event.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    print(df_match_event)
else:
    print("No data fetched.")

# print(df_match_statistics)
# if match_summary_data:
#     df_summary = pd.json_normalize(match_summary_data, sep='_')
#     df_summary.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#     print(df_summary)
# else:
#     print("No data fetched.")

Failed to fetch data for match ID kYcsmRc. Status code: 404
Failed to fetch data for match ID rYcsDPi. Status code: 404
Failed to fetch data for match ID Dwdbsbxpc. Status code: 404
Failed to fetch data for match ID tYkswXo. Status code: 404
Failed to fetch data for match ID nYcopYc. Status code: 404
Failed to fetch data for match ID aYcsYDwb. Status code: 404
Failed to fetch data for match ID zYcsLPi. Status code: 404
Failed to fetch data for match ID JZmsijab. Status code: 404
Failed to fetch data for match ID fOsGL. Status code: 404
Failed to fetch data for match ID iOmHO. Status code: 404
Failed to fetch data for match ID kOsBtg. Status code: 404
Failed to fetch data for match ID dibscXb. Status code: 404
Failed to fetch data for match ID NhbsUau. Status code: 404
Failed to fetch data for match ID Vhbsbvb. Status code: 404
Failed to fetch data for match ID habswob. Status code: 404
Failed to fetch data for match ID zobstnB. Status code: 404
Failed to fetch data for match ID fotsMLo

In [13]:
columns = df_match_event.columns.tolist()
print("Columns in df_match_event:")
for col in columns:
    print(col)

Columns in df_match_event:
match_id
status_code
customId
winnerCode
aggregatedWinnerCode
hasGlobalHighlights
hasEventPlayerStatistics
hasEventPlayerHeatMap
detailId
crowdsourcingDataDisplayEnabled
id
crowdsourcingEnabled
previousLegEventId
startTimestamp
slug
finalResultOnly
feedLocked
isEditor
tournament_name
tournament_slug
tournament_category_name
tournament_category_slug
tournament_category_sport_name
tournament_category_sport_slug
tournament_category_sport_id
tournament_category_id
tournament_category_country_alpha2
tournament_category_country_alpha3
tournament_category_country_name
tournament_category_flag
tournament_category_alpha2
tournament_uniqueTournament_name
tournament_uniqueTournament_slug
tournament_uniqueTournament_primaryColorHex
tournament_uniqueTournament_secondaryColorHex
tournament_uniqueTournament_category_name
tournament_uniqueTournament_category_slug
tournament_uniqueTournament_category_sport_name
tournament_uniqueTournament_category_sport_slug
tournament_unique

In [31]:
print(combined_df[combined_df["status_code"] == 404])

Empty DataFrame
Columns: [customId, winnerCode, hasGlobalHighlights, hasEventPlayerStatistics, hasEventPlayerHeatMap, detailId, crowdsourcingDataDisplayEnabled, match_id, crowdsourcingEnabled, startTimestamp, slug, finalResultOnly, feedLocked, isEditor, tournament_name, tournament_slug, tournament_category_name, tournament_category_slug, tournament_category_sport_name, tournament_category_sport_slug, tournament_category_sport_id, tournament_category_id, tournament_category_flag, tournament_uniqueTournament_name, tournament_uniqueTournament_slug, tournament_uniqueTournament_category_name, tournament_uniqueTournament_category_slug, tournament_uniqueTournament_category_sport_name, tournament_uniqueTournament_category_sport_slug, tournament_uniqueTournament_category_sport_id, tournament_uniqueTournament_category_id, tournament_uniqueTournament_category_flag, tournament_uniqueTournament_userCount, tournament_uniqueTournament_crowdsourcingEnabled, tournament_uniqueTournament_hasPerformanceGr

In [6]:
# customId = "xdbsEgb"
url = f"https://www.sofascore.com/api/v1/team/*"

response = requests.get(url)
status_code = response.status_code
print(status_code)
    

404


In [47]:
match_id = "12173508"
url = f"https://www.sofascore.com/api/v1/event/{match_id}/statistics"
try:
    response = requests.get(url)
    status_code = response.status_code
    if status_code == 200:
        data = response.json()
        # Assuming 'statistics' is a list containing match statistics
        for event in data['statistics']:
            for group in event['groups']:
                for stat_item in group['statisticsItems']:
                    # Append extracted data to the match_statistics_data list
                    match_statistics_data.append({
                        'period': event['period'],
                        'groupName': group['groupName'],
                        'name': stat_item['name'],
                        'home': stat_item['home'],
                        'away': stat_item['away'],
                        'compareCode': stat_item['compareCode'],
                        'statisticsType': stat_item['statisticsType'],
                        'valueType': stat_item['valueType'],
                        'homeValue': stat_item['homeValue'],
                        'awayValue': stat_item['awayValue'],
                        'renderType': stat_item['renderType'],
                        'key': stat_item['key']
                    })
    else:
        # Print an error message if the request fails
        print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
except requests.exceptions.RequestException as e:
    # Handle specific exceptions related to requests
    print(f"An error occurred for match ID {match_id}: {e}")

In [15]:
url = f"https://www.sofascore.com/api/v1/event/12173508/best-players/summary"
response = requests.get(url)
status_code = response.status_code
print(status_code)

200


In [19]:
df=pd.json_normalize(data)
print(df)

   confirmed                                       home.players  \
0       True  [{'player': {'name': 'Daniel Carvajal', 'slug'...   

  home.supportStaff home.formation home.playerColor.primary  \
0                []          4-4-2                   ffffff   

  home.playerColor.number home.playerColor.outline  \
0                  000000                   ffffff   

  home.playerColor.fancyNumber home.goalkeeperColor.primary  \
0                       222226                       4be438   

  home.goalkeeperColor.number  ... away.formation away.playerColor.primary  \
0                      ffffff  ...        4-2-3-1                   090606   

  away.playerColor.number away.playerColor.outline  \
0                  8d287e                   090606   

  away.playerColor.fancyNumber away.goalkeeperColor.primary  \
0                       ffffff                       f92b91   

  away.goalkeeperColor.number away.goalkeeperColor.outline  \
0                      cff149                  

In [40]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)